In [1]:
# load dependencies and data
import matplotlib.pyplot as plt
import pandas as pd


In [37]:
# found file type via osX  command: 
# ~/covid-viz diegoolano$ file -I county_popestimate2019.csv 
#                          county_popestimate2019.csv: text/plain; charset=iso-8859-1

# https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
# had to change county_popest to read 0's to front of some counties
pdata = pd.read_csv("county_popestimate2019.csv",encoding = "iso-8859-1", dtype = {"fips" : "string"})
hdata = pd.read_csv("us-counties0404.csv", dtype = {"fips" : "string"})

In [38]:
print(len(pdata))
pdata.head(3)

3193


,fips,state,County,pop2019
0,01000,Alabama,Alabama,4903185
1,01001,Alabama,Autauga County,55869
2,01003,Alabama,Baldwin County,223234


In [39]:
print(len(hdata))
hdata.head(3)

2408


,date,county,state,fips,cases,deaths
0,2020-04-04,Baldwin,Alabama,01003,29,1
1,2020-04-04,Barbour,Alabama,01005,2,0
2,2020-04-04,Bibb,Alabama,01007,4,0


In [ ]:
# TODOs datawise
#1. remove state level stuff from pdata  (ie, where state == County)
#2. combine hdata ["cases", deaths ] into pdata  
#3. add percap_cases, percap_deaths

In [50]:
# https://stackoverflow.com/questions/49063008/drop-rows-with-same-value-in-some-columns-in-dataframe-using-python
#pdata[pdata.iloc[:, 1:3].nunique(1) == 1]  .. includes District of Columbia 11001 mistakenly 

# instead do based on fips string
#pdata[pdata.fips.str.endswith("000")]   #these are all the states

In [51]:
#1. remove state rows
newd = pdata[~pdata.fips.str.endswith("000")]
print(len(newd), len(pdata))

3142 3193


In [71]:
#2. combine hdata into pdata based on fips
# https://datacarpentry.org/python-ecology-lesson/05-merging-data/  
hsub = hdata[["fips","cases","deaths"]]
ndf = pd.merge(newd, hsub, on=['fips'],  how='left')
len(ndf)

3142

In [72]:
ndf.head()

,fips,state,County,pop2019,cases,deaths
0,01001,Alabama,Autauga County,55869,NaN,NaN
1,01003,Alabama,Baldwin County,223234,29.0,1.0
2,01005,Alabama,Barbour County,24686,2.0,0.0
3,01007,Alabama,Bibb County,22394,4.0,0.0
4,01009,Alabama,Blount County,57826,10.0,0.0


In [73]:
# fill in NaNs with zeros !
ndf = ndf.fillna(0)

In [82]:
#3. add percap_cases, percap_deaths  per 100,000
ndf['cases_per_cap'] = round((ndf.cases / ndf.pop2019 ) * 100000,0)
ndf['deaths_per_cap'] = round((ndf.deaths / ndf.pop2019) * 100000,0)

In [83]:
ndf.head()

,fips,state,County,pop2019,cases,deaths,cases_per_cap,deaths_per_cap
0,01001,Alabama,Autauga County,55869,0.0,0.0,0.0,0.0
1,01003,Alabama,Baldwin County,223234,29.0,1.0,13.0,0.0
2,01005,Alabama,Barbour County,24686,2.0,0.0,8.0,0.0
3,01007,Alabama,Bibb County,22394,4.0,0.0,18.0,0.0
4,01009,Alabama,Blount County,57826,10.0,0.0,17.0,0.0


In [77]:
ndf.to_csv("covid0404countydata.csv", index=False)

In [ ]:
# TODOs on viz
# 1. make it so that you can filter to only shower bigger counties 
#  ( ie, those over a user defined threshold .. )